In [18]:
import kfp
import kfp.dsl as dsl
from kfp.dsl import component, Input, Output, InputPath, OutputPath, Dataset, Metrics, Model, Artifact, ClassificationMetrics, Markdown

# Load_Data Component

In [19]:
@component(
    packages_to_install=["tensorflow", "numpy"]
)
def load_data(
    dataset: Output[Dataset]
):
    import tensorflow as tf
    import numpy as np
    
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train = x_train / 255.0
    x_test = x_test / 255.0
    
    with open(dataset.path, "wb") as f:
        np.savez(
            f,
            x_train=x_train,
            y_train=y_train,
            x_test=x_test,
            y_test=y_test
        )

# Model_Building Component

In [20]:
@component(
    packages_to_install=["tensorflow", "numpy","scikit-learn"]
)
def model_building(
    no_epochs: int,
    lr: float,
    dataset: Input[Dataset],
    model: Output[Model],
    conf_matrix: Output[ClassificationMetrics],
    metrics: Output[Metrics],
    model_summary: Output[Markdown]
):
    """
    Build the model with Keras API
    Export model parameters
    """
    from tensorflow import keras
    import tensorflow as tf
    import numpy as np
    from sklearn.metrics import confusion_matrix
    
    with open(dataset.path, "rb") as f:
        dataset = np.load(f)
        x_train, y_train = dataset["x_train"], dataset["y_train"]
        x_test, y_test = dataset["x_test"], dataset["y_test"]
    
    
    train_model = keras.models.Sequential()
    train_model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28,28,1)))
    train_model.add(keras.layers.MaxPool2D(2, 2))

    train_model.add(keras.layers.Flatten())
    train_model.add(keras.layers.Dense(64, activation='relu'))

    train_model.add(keras.layers.Dense(32, activation='relu'))

    train_model.add(keras.layers.Dense(10, activation='softmax')) #output are 10 classes, numbers from 0-9
    
    
    #compile the model - we want to have a binary outcome

    optimizer = keras.optimizers.Adam(learning_rate=lr)

    train_model.compile(optimizer=optimizer,
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])
    
    #fit the model and return the history while training
    history = train_model.fit(
      x=x_train,
      y=y_train,
      epochs=no_epochs,
      batch_size=20,
    )
    

    # Test the model against the test dataset
    # Returns the loss value & metrics values for the model in test mode.
    model_loss, model_accuracy = train_model.evaluate(x=x_test,y=y_test)
    
    predictions = train_model.predict(x_test)

    confusion = confusion_matrix(y_test,np.argmax(predictions,axis=1)).tolist()
    
    conf_matrix.log_confusion_matrix(
        ['0','1','2','3','4','5','6','7','8','9'],
        confusion_matrix(y_test,np.argmax(predictions,axis=1)).tolist() # .tolist() to convert np array to list.
    )   
    
    metrics.log_metric("accuracy", (model_accuracy*100))
    
    stringlist = []
    train_model.summary(print_fn=lambda x: stringlist.append(x))
    markdown_content = "\n".join(stringlist)
    with open(model_summary.path,'w') as f:
        f.write(markdown_content)
    
    
    with open(model.path,'wb') as f:
        train_model.save(f)

    
    

# Test Component - Model 못불러옴

In [24]:
@component(
    packages_to_install=["tensorflow", "numpy","scikit-learn"]
)
def test(
    dataset: Input[Dataset],
    model: Input[Model],
    metrics: Output[Metrics]
):
    from tensorflow import keras
    import tensorflow as tf
    import numpy as np
    
    
    with open(dataset.path, "rb") as f:
        dataset = np.load(f)
        x_train, y_train = dataset["x_train"], dataset["y_train"]
        x_test, y_test = dataset["x_test"], dataset["y_test"]
    
    test_model = tf.keras.models.load_model(model.path+'/model.h5')
    model_loss, model_accuracy = test_model.evalutate(x=x_test,y=y_test)
    
    metrics.log_metric("accuracy", (model_accuracy*100))

In [25]:
#dsl.get_pipeline_conf().add_op_transformer(aws.use_aws_secret(secret_name=''))

# Pipeline Compile

In [26]:
@dsl.pipeline(
    name="tensorflow-v2-I/O",
    #pipeline_root="s3://krcloud.s3.hmckmc.co.kr/aipsystemdev"
)
def pipeline(
    no_epochs: int,
    lr: float
):
    download_data_task = load_data()
 
    train_task = model_building(no_epochs=1,lr=0.001,dataset=download_data_task.output)
    #test_task = test(dataset=download_data_task.output,model=train_task.outputs['model'])

kfp.compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path='tesnorflow_v2.yaml')


# Client 연동

In [27]:
import requests

USERNAME = "user@example.com"
PASSWORD = "12341234"
NAMESPACE = "kubeflow-user-example-com"
HOST = "http://172.17.0.7:8080" # istio-ingressgateway pod ip:port

session = requests.Session()
response = session.get(HOST)

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {"login": "user@example.com", "password": "12341234"}
session.post(response.url, headers=headers, data=data)
session_cookie = session.cookies.get_dict()["authservice_session"]

client = kfp.Client(
    host=f"{HOST}/pipeline",
    namespace=f"{NAMESPACE}",
    cookies=f"authservice_session={session_cookie}",
)

/opt/conda/lib/python3.8/site-packages/kfp/client/client.py:140: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-alpha.0 and later versions.
  warnings.warn(


# Run Pipeline

In [28]:
arguments = {
    'no_epochs':10,
    'lr':0.001
}
client.create_run_from_pipeline_func(pipeline_func=pipeline,arguments=arguments,enable_caching=False)

RunPipelineResult(run_id=54f3205c-294e-49e9-bbd6-b845696eecea)